<a href="https://colab.research.google.com/github/map72ru/data_mining/blob/main/hw_webinar_6_with_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606593 sha256=0bd5401f1544e1cd1d11350ff3930dc2518ee729790190734cb91518db83dedc
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/hw2')

import sys
sys.path.append('/content/hw2/MyDrive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/hw2


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [4]:
data = pd.read_csv('/content/hw2/MyDrive/data/retail_train.csv')
item_features = pd.read_csv('/content/hw2/MyDrive/data/product.csv')
user_features = pd.read_csv('/content/hw2/MyDrive/data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [6]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [7]:
recommender.model.user_factors[2369]

Matrix([[0.825682   0.5541909  0.5044843  0.6543931  0.42744341 0.8411518
  0.72042966 0.6885215  0.9596684  0.8997363  0.5450086  0.9206787
  0.53386724 0.18284307 0.6701355  0.3945965  0.4660829  0.7126198
  0.04154523 0.24131866]])

In [8]:
recommender.get_als_recommendations(2375, N=5)

ValueError: ignored

In [9]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [10]:
recommender.get_similar_items_recommendation(2375, N=5)

[117847, 117847, 117847, 117847, 117847]

In [13]:
recommender.user_item_matrix.getrow(recommender.userid_to_id[2375])

<1x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 124 stored elements in Compressed Sparse Row format>

In [11]:
recommender.get_similar_users_recommendation(2375, N=5)

ValueError: ignored

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [ ]:
userid = 2372
k = 50

user_items_id = data_val_lvl_1.query('user_id == @userid').item_id.unique()

recomended_items_id = recommender.get_als_recommendations(userid, N=k)

als_recall = recall_at_k(recomended_items_id, user_items_id, k)
print(f"als={als_recall}")

recomended_items_id = recommender.get_own_recommendations(userid, N=k)
own_recall = recall_at_k(recomended_items_id, user_items_id, k)
print(f"own={own_recall}")

recomended_items_id = recommender.get_similar_items_recommendation(userid, N=k)
sitems_recall = recall_at_k(recomended_items_id, user_items_id, k)
print(f"similar_items={sitems_recall}")

recomended_items_id = recommender.get_similar_users_recommendation(userid, N=k)
susers_recall = recall_at_k(recomended_items_id, user_items_id, k)
print(f"similar_users={susers_recall}")

als=5.88235294117647

own=8.823529411764707

similar_items=4.411764705882353

similar_users=0.0


own+top popular дают лучший результат

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}

C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

In [ ]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [ ]:
for m in [20, 50, 100, 200, 500]:
    recomended_items_id = recommender.get_own_recommendations(userid, N=m)
    own_recall = 100*recall_at_k(recomended_items_id, user_items_id, m)
    print(f"k = {m} own={own_recall}")

k = 20 own=1.4705882352941175

k = 50 own=8.823529411764707

k = 100 own=13.23529411764706

k = 200 own=17.647058823529413

k = 500 own=27.941176470588236


Чем больше k, тем метрика будет больше, поскольку вероятность попадания рекоментованных в купленные товары будет возростать. Если разумность значения k рассматривать чисто с математической точки зрения, то при достаточно большом K мы будем увеличивать метрику до 1. Если "разумность" рассмтаривать с точки зрения взаимодействия с пользователем, то самым разумным будет 20 из предложенного списка, хотя recall равен 1,47. Видно так же, что после 200 прирост точности замедляется. Я делал расчте для 1000. recall@1000 был равен 27.941176470588236


### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
# your_code

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 